In [14]:
import networkx as nx
import sys
sys.path.append(r'c:\\Users\\rotim\\OneDrive\\Documents\\Reading\\graph-code\\Skoltech-PhD-Thesis\\_codes\\SIR\\Influential-nodes-identification-in-complex-networks-via-information-entropy')
from algorithms import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle

In [15]:
data_file = 'topo'  # 'HepPh'
G = clean_data(data_file)

In [16]:
print(nx.number_of_nodes(G), nx.number_of_edges(G))

34 42


In [17]:
geo_loc_data = get_geo_data("Internet2LatLong.csv")
edge_geo_data_combined = assign_location(G, geo_loc_data)
# set edge attributes
set_edge_attr(G, edge_geo_data_combined)
# obtain the node degrees and edges weights to desired lengths
tmp_t = list(range(1, 5))
tmp_t_SN, tmp_t_hub = hubs_SN_NS(G, tmp_t)
# unpack the nodes and edges
tmp_t_SN_1, tmp_t_SN_2, tmp_t_SN_3, tmp_t_SN_4 = tmp_t_SN
tmp_t_hub_1, tmp_t_hub_2, tmp_t_hub_3, tmp_t_hub_4 = tmp_t_hub
# determine variables values
k_max, k_min, k_2_max, k_2_min, sigma, delta = maxi_mini(
    tmp_t_SN_1[1], tmp_t_hub_2[2])
# assign various probability weights
w_d_h, w_d_2_h, w_d_l, w_d_2_l, w_d_t, w_d_2_t = probability_weights(
    tmp_t_SN_1[1], tmp_t_hub_2[2], k_max, k_min, k_2_max, k_2_min, sigma, delta)
# combine into a dictionary
combined_dict, combined_dict_k_2 = covert_to_dict(
    w_d_h, w_d_l, w_d_t), covert_to_dict(w_d_2_h, w_d_2_l, w_d_2_t)
# print(combined_dict, "\n ...... \n", combined_dict_k_2)

# Get ranked result
opti_rank, ranked_nodes = rank_result(combined_dict, combined_dict_k_2)

print(opti_rank)


[('27', 0.9985005730418679), ('5', 0.7549436527724699), ('12', 0.7543488781301699), ('28', 0.6538494099071029), ('2', 0.5865976594398602)]


In [13]:
r = [0.8, 0.1, 0.4, 0.2, 0.3, 0.5]
topk_list = [round(nx.number_of_nodes(G) * k) for k in r]
print(topk_list)

[27, 3, 14, 7, 10, 17]


In [ ]:
max_topk = round(max(topk_list) * nx.number_of_nodes(G))
newmethod_1_rank = EnRenewRank(G, max_topk, 1)
print('done')
newmethod_2_rank = EnRenewRank(G, max_topk, 2)
print('done')
newmethod_3_rank = EnRenewRank(G, max_topk, 3)
print('done')
newmethod_4_rank = EnRenewRank(G, max_topk, 4)
print('done')


In [ ]:
infect_prob = compute_probability(G) * 1.5
atio = 1.5
cover_prob = infect_prob / atio
avg = 100
max_iter = 200000

## Different order

In [ ]:
%%time
newmethod_1_rankresult = []
newmethod_2_rankresult = []
newmethod_3_rankresult = []
newmethod_4_rankresult = []


for k in tqdm(topk_list):
    topk = k
    newmethod_1_rankresult.append(get_sir_result(G, newmethod_1_rank, topk, avg, infect_prob, cover_prob, max_iter)[-1] / nx.number_of_nodes(G))
    newmethod_2_rankresult.append(get_sir_result(G, newmethod_2_rank, topk, avg, infect_prob, cover_prob, max_iter)[-1] / nx.number_of_nodes(G))
    newmethod_3_rankresult.append(get_sir_result(G, newmethod_3_rank, topk, avg, infect_prob, cover_prob, max_iter)[-1] / nx.number_of_nodes(G))
    newmethod_4_rankresult.append(get_sir_result(G, newmethod_4_rank, topk, avg, infect_prob, cover_prob, max_iter)[-1] / nx.number_of_nodes(G))

In [ ]:
%matplotlib notebook
plt.title('network:{}   avg:{}  i/r:{}   infect_prob:{}'.format(data_file, avg, atio, infect_prob))
plt.plot(np.array(topk_list) / nx.number_of_nodes(G), newmethod_1_rankresult, 'r-o', label='order=1')
plt.plot(np.array(topk_list) / nx.number_of_nodes(G), newmethod_2_rankresult, 'b-o', label='order=2')
plt.plot(np.array(topk_list) / nx.number_of_nodes(G), newmethod_3_rankresult, 'y-o', label='order=3')
plt.plot(np.array(topk_list) / nx.number_of_nodes(G), newmethod_4_rankresult, 'g-o', label='order=4')
plt.legend()
plt.show()